In [23]:
# 칼럼명 고정
order = '온라인 주문번호'
count = '수량'
sku = '품번'

# 슈트 개수
shut = 60

In [3]:
import pandas as pd
df = pd.read_excel('data/BA_Sample_Samsung_0421_raw_data.xlsx',usecols= [order,count,sku])
df

,품번,수량,온라인 주문번호
0,111721BPAA070,1,OD202204211595605
1,111871LY6RM,1,OD202204211595605
2,121771DYAAM,1,OD202204211595605
3,321751AY1LM,1,OD202204211595605
4,321751LY2PM,1,OD202204211595605
...,...,...,...
14534,192196WY75F,1,OD202204201590292
14535,321821LH8KM,5,OD202204201590292
14536,351525JQ2E070,1,OD202204201590292
14537,351565JQ2EM,1,OD202204201590292


#### 알고리즘1: 모든 슈트 해방 후 일관되게 주문 처리 방식


1. 주문번호끼리 그룹화

2. 작업량 = 품번의 개수 * 수량 -> 그룹(주문번호)마다 작업량 계산

3. 작업량이 많은 순으로 정렬(내림차순)(=투입순서)

4. 각 슈트에는 아래와 같이 할당됨(숫자가 작을수록 우선순위, 작업량 많은 항목)(ㄹ모양)  
    <0> <1>          <58> <59>  
0-   0   1    ....    58   59   ->   1번 배치 (0\*60+i+1)   ->   y-0\*60  
1-  119 118   ....    61   60   ->   2번 배치 (2\*60-i)     ->   2\*60-1-y  
2-  120 121   ....   178  179   ->   3번 배치 (2\*60+i+1)   ->   y-2\*60

#### 생성결과

1. 투입 순서 + 배치 순서 

2. 배당되는 슈트

In [21]:
order_df = df.groupby(order)[count].sum().reset_index()
order_df

,온라인 주문번호,수량
0,OD202204091413193,3
1,OD202204121453868,2
2,OD202204131475710,3
3,OD202204131476387,2
4,OD202204131480783,2
...,...,...
3111,OD202204211607081,11
3112,OD202204211607083,6
3113,OD202204211607084,3
3114,OD202204211607085,5


In [22]:
order_df = order_df.sort_values(by=count, ascending=False).reset_index(drop=True)
order_df

,온라인 주문번호,수량
0,OD202204211605517,69
1,OD202204211596980,45
2,OD202204211596072,37
3,OD202204211604990,34
4,OD202204201592352,32
...,...,...
3111,OD202204211604937,1
3112,OD202204201591190,1
3113,OD202204211601155,1
3114,OD202204201590428,1


In [32]:
order_df['투입순서'] = order_df.index
order_df['배치'] = order_df.index // shut
order_df

,온라인 주문번호,수량,투입순서,배치
0,OD202204211605517,69,0,0
1,OD202204211596980,45,1,0
2,OD202204211596072,37,2,0
3,OD202204211604990,34,3,0
4,OD202204201592352,32,4,0
...,...,...,...,...
3111,OD202204211604937,1,3111,51
3112,OD202204201591190,1,3112,51
3113,OD202204211601155,1,3113,51
3114,OD202204201590428,1,3114,51


In [41]:
for i in range(len(order_df)):
    if (order_df.loc[i, '배치'] % 2 == 1):
        order_df.loc[i, '슈트'] = shut*(order_df.loc[i, '배치']+1)-i-1
    else:
        order_df.loc[i, '슈트'] = i- order_df.loc[i, '배치']*shut

order_df.head(62)

,온라인 주문번호,수량,투입순서,배치,슈트
0,OD202204211605517,69,0,0,0.0
1,OD202204211596980,45,1,0,1.0
2,OD202204211596072,37,2,0,2.0
3,OD202204211604990,34,3,0,3.0
4,OD202204201592352,32,4,0,4.0
...,...,...,...,...,...
57,OD202204201590275,16,57,0,57.0
58,OD202204211595413,16,58,0,58.0
59,OD202204211602732,16,59,0,59.0
60,OD202204211597127,15,60,1,59.0


In [42]:
print(order_df.loc[118, '슈트'])
print(order_df.loc[119, '슈트'])
print(order_df.loc[120, '슈트'])
print(order_df.loc[121, '슈트'])
print(order_df.loc[122, '슈트'])

1.0
0.0
0.0
1.0
2.0


In [43]:
order_df

,온라인 주문번호,수량,투입순서,배치,슈트
0,OD202204211605517,69,0,0,0.0
1,OD202204211596980,45,1,0,1.0
2,OD202204211596072,37,2,0,2.0
3,OD202204211604990,34,3,0,3.0
4,OD202204201592352,32,4,0,4.0
...,...,...,...,...,...
3111,OD202204211604937,1,3111,51,8.0
3112,OD202204201591190,1,3112,51,7.0
3113,OD202204211601155,1,3113,51,6.0
3114,OD202204201590428,1,3114,51,5.0


In [47]:
workload = []
for s in range(shut):
    workload.append(order_df[order_df['슈트']==s]['수량'].sum())
print(len(workload))
print(workload)

60
[293, 269, 260, 257, 256, 254, 252, 251, 249, 250, 248, 247, 247, 247, 249, 248, 248, 247, 247, 247, 247, 247, 247, 247, 246, 246, 247, 246, 246, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 246, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 246, 246, 246, 246, 246, 247]


#### 알고리즘2: 하나의 슈트가 해방될 경우 해당 슈트는 다음 주문 처리


1. 주문번호끼리 그룹화

2. 작업량 = 품번의 개수 * 수량 -> 그룹(주문번호)마다 작업량 계산

3. 총작업량 계산 후 각 슈트당 작업량 할당

4. 작업을 남아있는 작업량이 가장 많은 슈트에 할당, 남아있는 작업량 업데이트

In [48]:
order_df = df.groupby(order)[count].sum().reset_index()
#총 작업량
print(order_df['수량'].sum())

14914


In [49]:
#슈트당 작업량 할당
current_workload = [order_df['수량'].sum()/shut]*shut
current_workload

[248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666666666,
 248.56666666

In [50]:
for i in range(len(order_df)):
    index = current_workload.index(max(current_workload))
    order_df.loc[i, '슈트'] = index
    current_workload[index] -= order_df.loc[i, count]

order_df

,온라인 주문번호,수량,슈트
0,OD202204091413193,3,0.0
1,OD202204121453868,2,1.0
2,OD202204131475710,3,2.0
3,OD202204131476387,2,3.0
4,OD202204131480783,2,4.0
...,...,...,...
3111,OD202204211607081,11,7.0
3112,OD202204211607083,6,8.0
3113,OD202204211607084,3,9.0
3114,OD202204211607085,5,18.0


In [51]:
workload = []
for s in range(shut):
    workload.append(order_df[order_df['슈트']==s]['수량'].sum())
print(len(workload))
print(workload)

60
[246, 254, 249, 246, 246, 251, 246, 255, 250, 247, 245, 251, 245, 247, 256, 247, 246, 291, 249, 251, 245, 251, 244, 244, 244, 244, 247, 245, 244, 257, 245, 250, 247, 246, 249, 246, 244, 262, 249, 244, 258, 248, 247, 249, 246, 248, 246, 245, 244, 247, 246, 246, 247, 245, 247, 245, 247, 254, 245, 249]
